In [1]:
import os
import sys
sys.path.append('..')
from backtest.operators import * 
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

### Get Weighting

##### 尚須新增過去幾天參數

In [2]:
collection = "monthly_revenue"
mv_pstage = TWMarketMonitor().get_data_from_mongo(database_str= 'PSTAGE',
                                                  collection_str = collection,
                                                  start_date = "2020-01-01"
                                                  )
mv_pstage

TypeError: TWMarketMonitor.get_data_from_mongo() got an unexpected keyword argument 'limit'

In [ ]:
mv_pstage

""


In [ ]:
collection = "monthly_revenue"
monthly_rev_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [ ]:
monthly_rev_pmart

{'去年單月營收(千元)': symbol        000930    000960     0009A0       1101       1102  1103  1104  \
 datetime                                                                      
 1990-01-04       NaN       NaN        NaN        NaN        NaN   NaN   NaN   
 1990-01-05       NaN       NaN        NaN        NaN        NaN   NaN   NaN   
 1990-01-06       NaN       NaN        NaN        NaN        NaN   NaN   NaN   
 1990-01-08       NaN       NaN        NaN        NaN        NaN   NaN   NaN   
 1990-01-09       NaN       NaN        NaN        NaN        NaN   NaN   NaN   
 ...              ...       ...        ...        ...        ...   ...   ...   
 2024-03-29  360885.0  999390.0  1119497.0  7306069.0  6241236.0   NaN   NaN   
 2024-04-01  360885.0  999390.0  1119497.0  7306069.0  6241236.0   NaN   NaN   
 2024-04-02  360885.0  999390.0  1119497.0  7306069.0  6241236.0   NaN   NaN   
 2024-04-03  360885.0  999390.0  1119497.0  7306069.0  6241236.0   NaN   NaN   
 2024-04-08  426304.0     

In [ ]:
monthly_rev_pmart['單月營收(千元)']#['revenue_yoy']['2023-12-01':]

symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,0.614813,1.087764,0.894899,-0.175610,-0.117737,0.228784,0.030762,0.124486,0.008976,-0.032377,...,0.297714,0.173870,-0.343062,1.066110,0.138080,0.239030,NaN,0.419100,0.019849,-0.235536
2023-12-04,0.614813,1.087764,0.894899,-0.175610,-0.117737,0.228784,0.030762,0.124486,0.008976,-0.032377,...,0.297714,0.173870,-0.343062,1.066110,0.138080,0.239030,NaN,0.419100,0.019849,-0.235536
2023-12-05,0.614813,1.087764,0.894899,-0.175610,-0.117737,0.228784,0.030762,0.124486,0.008976,-0.032377,...,0.297714,0.173870,-0.343062,1.066110,0.138080,0.239030,NaN,0.419100,0.019849,-0.235536
2023-12-06,0.614813,1.087764,0.894899,-0.175610,-0.117737,0.228784,0.030762,0.124486,0.008976,-0.032377,...,0.297714,0.173870,-0.343062,1.066110,0.138080,0.239030,NaN,0.419100,0.019849,-0.235536
2023-12-07,0.614813,1.087764,0.894899,-0.175610,-0.117737,0.228784,0.030762,0.124486,0.008976,-0.032377,...,0.297714,0.173870,-0.343062,1.066110,0.138080,0.239030,NaN,0.419100,0.019849,-0.235536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,0.293030,0.603638,0.130549,-0.343206,-0.423193,-0.046221,-0.193010,-0.208463,-0.448187,0.006145,...,-0.409339,-0.957993,-0.109271,0.222095,-0.104021,-0.087035,-0.461333,0.026515,-0.235097,-0.520450
2024-04-01,0.293030,0.603638,0.130549,-0.343206,-0.423193,-0.046221,-0.193010,-0.208463,-0.448187,0.006145,...,-0.409339,-0.957993,-0.109271,0.222095,-0.104021,-0.087035,-0.461333,0.026515,-0.235097,-0.520450
2024-04-02,0.293030,0.603638,0.130549,-0.343206,-0.423193,-0.046221,-0.193010,-0.208463,-0.448187,0.006145,...,-0.409339,-0.957993,-0.109271,0.222095,-0.104021,-0.087035,-0.461333,0.026515,-0.235097,-0.520450


In [ ]:
monthly_rev_pmart['revenue_yoy'] = monthly_rev_pmart['單月營收(千元)'] / monthly_rev_pmart['去年單月營收(千元)'] -1
example_factor = ts_rank(monthly_rev_pmart['revenue_yoy'],252)
example_factor

symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_weighting(factor):
    demean = factor.sub(factor.mean(axis=1), axis=0)
    weighting = demean.div(demean.abs().sum(axis=1), axis=0)
    return weighting

### 假設出訊號後隔天收盤進場

In [ ]:
example_signal = get_weighting(example_factor)
example_weighting = example_signal.shift(2) #隔天收盤進場，後天才有部位
example_weighting

symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get returns

In [ ]:
collection = 'securities_returns'
returns_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [ ]:
returns_pmart

{'日報酬率(%)': symbol      0015    0029    0050    0051    0052    0053    0055    0056  \
 datetime                                                                   
 2024-04-01   0.0 -0.9422 -0.3167  0.4456 -0.3139 -0.6407 -0.2364 -0.6774   
 2024-04-02   0.0  1.2581  1.4295  0.0000  2.0151  1.8757 -0.2765  0.7325   
 2024-04-03   0.0  3.0303 -0.6890 -0.1267 -0.7716 -0.2877 -0.8713 -0.3260   
 2024-04-08   0.0 -0.7941  0.5676  0.4442  0.2177  0.1731  0.7990  0.4277   
 
 symbol        0057    0061  ...  IX0059  IX0060  IX0075  IX0085  IX0099  \
 datetime                    ...                                           
 2024-04-01 -0.5991  1.7762  ...  0.0191     NaN -0.1288     NaN  2.5392   
 2024-04-02  1.3775 -0.3490  ... -0.1669  0.4918  1.6493  0.6206  1.8698   
 2024-04-03 -0.4246 -0.5254  ... -0.1528  0.1224 -0.5212 -0.2409  1.5316   
 2024-04-08  0.4691 -0.1174  ...  0.3253  0.1955 -2.9432 -0.2318  1.6824   
 
 symbol      IX0118    MSCI  T1001Y  T1010Y  T1012Y  
 datetime    

In [ ]:
collection = 'securities_trading_data'
ohlcv_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [ ]:
ohlcv_pmart['開盤價']

symbol,0050,0051,0052,0053,0055,0056,0057,0061,006201,006203,...,9946,9949,9950,9951,9955,9957,9958,9960,9962,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-29,NaN,NaN,158.45,NaN,NaN,39.75,116.85,16.92,21.96,75.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.90,NaN
2024-04-01,158.20,78.7,160.00,85.95,25.38,39.89,117.85,16.90,22.25,75.90,...,25.50,18.6,17.00,72.4,25.15,6.44,220.0,27.1,18.10,NaN
2024-04-02,158.45,78.9,159.25,86.80,25.32,39.62,117.50,17.19,22.25,75.50,...,26.25,18.8,17.50,72.7,25.60,6.40,218.5,27.0,18.15,NaN
2024-04-03,158.65,78.8,161.20,85.70,25.25,39.69,117.25,17.23,22.18,75.75,...,26.70,18.8,17.30,72.6,26.00,6.38,213.5,26.9,18.30,NaN
2024-04-08,158.65,78.8,161.00,86.95,25.03,39.77,117.45,17.04,22.27,75.95,...,26.45,19.1,17.35,72.5,25.55,6.84,216.0,26.9,18.15,63.5


In [ ]:
example_returns = returns_pmart['日報酬率(%)']*0.01
example_returns

symbol,0015,0029,0050,0051,0052,0053,0055,0056,0057,0061,...,IX0059,IX0060,IX0075,IX0085,IX0099,IX0118,MSCI,T1001Y,T1010Y,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2024-04-01,0.0,-0.009422,-0.003167,0.004456,-0.003139,-0.006407,-0.002364,-0.006774,-0.005991,0.017762,...,0.000191,NaN,-0.001288,NaN,0.025392,NaN,NaN,0.0,-0.009422,0.0
2024-04-02,0.0,0.012581,0.014295,0.000000,0.020151,0.018757,-0.002765,0.007325,0.013775,-0.003490,...,-0.001669,0.004918,0.016493,0.006206,0.018698,0.002900,0.016478,0.0,0.012581,0.0
2024-04-03,0.0,0.030303,-0.006890,-0.001267,-0.007716,-0.002877,-0.008713,-0.003260,-0.004246,-0.005254,...,-0.001528,0.001224,-0.005212,-0.002409,0.015316,0.000537,-0.009002,0.0,0.030303,0.0
2024-04-08,0.0,-0.007941,0.005676,0.004442,0.002177,0.001731,0.007990,0.004277,0.004691,-0.001174,...,0.003253,0.001955,-0.029432,-0.002318,0.016824,0.002674,0.003356,0.0,-0.007941,0.0


### 將weighting 與 returns結合獲得績效

In [ ]:
stocks = 20
top = example_weighting.apply(lambda row: row.nlargest(stocks), axis=1).fillna(0)
example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)
recent_w_weighting = example_equalweight#[-5:].dropna(axis =1, how = 'all')
recent_w_weighting = recent_w_weighting.loc[:, (recent_w_weighting != 0.0).any(axis=0)]
recent_w_weighting

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_1596/681453456.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)


symbol
datetime
1990-01-04
1990-01-05
1990-01-06
1990-01-08
1990-01-09
...
2024-03-29
2024-04-01
2024-04-02


In [ ]:
recent_w_returns = example_returns * recent_w_weighting
recent_w_profit = recent_w_returns.dropna(axis =1, how='all').dropna(axis=0, how='all')
recent_w_profit

symbol
datetime


In [ ]:
recent_w_profit.sum(axis =1)#.sum()

Series([], dtype: float64)